In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.animation import FuncAnimation, FFMpegWriter
from itertools import product
from IPython.display import HTML

plt.rcParams['animation.ffmpeg_path'] = 'C:/ffmpeg/bin/ffmpeg'

In [2]:
N=200
mass=1
radius=0.1
L=20
v_0=3
duration=10
dt=0.008
steps=int(duration/dt)

In [3]:
def collisions(position, velocity, radius, L, dt, N):
    r_next = position + radius * dt

    velocity[r_next[:, 0] < radius, 0] *= -1
    velocity[r_next[:, 0] > L - radius, 0] *= -1
    velocity[r_next[:, 1] < radius, 1] *= -1
    velocity[r_next[:, 1] > L - radius, 1] *= -1

    for i in range(N):
        for j in range(i + 1, N):
            if np.linalg.norm(r_next[i] - r_next[j]) < 2 * radius:
                delta_velocity = velocity[i] - velocity[j]
                delta_r = position[i] - position[j]

                velocity[i] -= delta_r.dot(delta_velocity) / (delta_r.dot(delta_r)) * delta_r
                velocity[j] += delta_r.dot(delta_velocity) / (delta_r.dot(delta_r)) * delta_r

def step(position, velocity, radius, L, dt, N):
    collisions(position, velocity, radius, L, dt, N)
    position += velocity * dt
    #Redondance

def simulation(position, velocity, radius, L, dt, N, steps):
    positions = np.zeros((steps, N, 2))
    velocities = np.zeros((steps, N))

    for n in range(steps):
        step(position, velocity, radius, L, dt, N)
        positions[n, :, :] = position
        velocities[n, :] = np.linalg.norm(velocity, axis=1)

    return positions, velocities

def update(frame):
    ax1.clear()
    for i in range(N):
        x, y = positions[frame,i, 0], positions[frame, i, 1]
        circle = plt.Circle((x, y), radius, fill=True)
        ax1.add_artist(circle)

    ax1.set_xlabel('$x$', fontsize=15)
    ax1.set_ylabel('$y$', fontsize=15)
    ax1.set_title('Ideal gas animation', fontsize=15)
    ax1.set_xlim(0, L)
    ax1.set_ylim(0, L)
    ax1.set_aspect('equal')
    ax1.set_xticks([]) #remove ticks
    ax1.set_yticks([])
    plt.tight_layout()

In [4]:
grid=int(np.ceil(np.sqrt(N)))
space=L/grid
x=np.linspace(radius+space/2, L-radius - space/2, grid)
pos=list(product(x, x))

position=np.array(pos[:N])

theta=np.random.uniform(0, 2*np.pi, size=N)
v_x,v_y=v_0*np.cos(theta), v_0*np.sin(theta)
velocity=np.stack((v_x,v_y), axis=1)

positions, velocities = simulation(position, velocity, radius, L, dt, N, steps)

In [ ]:
fig,ax1= plt.subplots(1, figsize = (12,6))

interval = duration*1e3/steps
animation = FuncAnimation(fig, update, frames=steps, interval=interval)
HTML(animation.to_html5_video())